# 두 문장 관계 분류를 위한 학습 데이터 구축

1. paraphrase를 detection할 수 있는 학습 데이터를 구축
2. 이 데이터를 통해서 두 문장 관계 분류를 학습

* 목적 : paraphrase이 된 데이터과 paraphrase가 되지 않은 데이터 구축

* paraKQC 데이터
  * 하나의 문장에 대해 10개의 유사한 paraphrasing된 문장을 가짐

* 데이터 다운로드 및 관찰

In [ ]:
!git clone https://github.com/warnikchow/paraKQC.git

In [ ]:
data = open('/content/paraKQC/data/paraKQC_v1.txt')

In [ ]:
lines = data.readlines()

In [ ]:
for i in range(0,15):
    print(lines[i]) # 10개씩 paraphasing 

'''

0	0	메일을 다 비울까 아니면 안읽은 것만 지울까?

0	0	메일 중에 안읽은 것만 지울까? 다 지울까?

0	0	안읽은 메일만 지워 다지워?

0	0	다 지울까 안읽은 메일만 지울까?

0	0	전체를 비울까 안읽은 것만 비울까?

0	0	안읽은 메일만 지울꺼야? 아니면 다 지울꺼야?

0	0	어떻게 지울까? 안읽은거만? 전체 다?

0	0	메일을 다 지울지 안읽은거만 지울지 알려주세요

0	0	메일은 다 지울수도 있고, 안읽은거만 지울 수도 있어. 어떻게 할래?

0	0	안읽은 메일만 지우든가, 다 지울 수 있는데 어떻게 할꺼야?

0	0	지메일 쓸래, 네이버 메일 쓸래

0	0	지메일을 쓸거야 네이버 메일을 쓸꺼야?

0	0	지메일, 네이버 둘 중에 뭘 쓸래?

0	0	네이버랑 지메일이 있는데 뭘 쓸래?

0	0	네이버랑 지메일 중에 골라줄래?
'''

In [ ]:
similar_sents = {}

* 10개씩 데이터 형태로 묶음
  * 전체 문장을 하나씩 읽다가 10개를 읽는 순간 데이터 배열로 10개의 말뭉치를 뺌

* `total_sent` : 전체 문장을 저장한 list
* `similar_sent` : paraphrasing된 묶음(10개씩 묶음)을 저장한 list

In [ ]:
similar_sent = []
total_sent = []
for line in lines:
    line = line.strip()
    sent = line.split('\t')[2]
    total_sent.append(sent)
    similar_sent.append(sent)
    if len(similar_sent) == 10:
        similar_sents[similar_sent[0]] = similar_sent[1:]
        similar_sent = []

In [ ]:
print(len(total_sent))  # 가장 유사한 문장을 찾기 위한 전체 문장 pool
# 10000

In [ ]:
for i in range(0,15):
    print(total_sent[i])

'''
메일을 다 비울까 아니면 안읽은 것만 지울까?
메일 중에 안읽은 것만 지울까? 다 지울까?
안읽은 메일만 지워 다지워?
다 지울까 안읽은 메일만 지울까?
전체를 비울까 안읽은 것만 비울까?
안읽은 메일만 지울꺼야? 아니면 다 지울꺼야?
어떻게 지울까? 안읽은거만? 전체 다?
메일을 다 지울지 안읽은거만 지울지 알려주세요
메일은 다 지울수도 있고, 안읽은거만 지울 수도 있어. 어떻게 할래?
안읽은 메일만 지우든가, 다 지울 수 있는데 어떻게 할꺼야?
지메일 쓸래, 네이버 메일 쓸래
지메일을 쓸거야 네이버 메일을 쓸꺼야?
지메일, 네이버 둘 중에 뭘 쓸래?
네이버랑 지메일이 있는데 뭘 쓸래?
네이버랑 지메일 중에 골라줄래?
'''

In [ ]:
print(len(similar_sents)) # 999

* 10개의 문장 중 첫 번째 문장을 key, 나머지 문장을 value로 저장

In [ ]:
for i, key in enumerate(similar_sents.keys()):  # 10개의 문장 중, 첫 번째 문장을 key
    print('\n', key)                            # 나머지 9개의 문장을 value
    for sent in similar_sents[key]:             # 헷갈리니까 이걸 similar_sents dict라고 정의할게요 :-)
        print("-", sent)
    if i > 3:
        break

* paraphrasing되지 않은 데이터를 구축해야함
  * 쉬운 문제 생성
    * total_sent 에서 random으로 서로 다른 문장을 선택함
    * 의미가 너무 상반된 결과를 가지고 학습하게 됨
  * 어려운 문제 생성
    * key 문장을 BERT를 통해 sentence embedding함
    * total_sent에 있는 문장을 전부 sentence embedding을 한 후 key 문장의 sentence embedding 결과와 가장 유사한 sentence embedding 결과를 가진 문장을 선택함
    * sentence embedding은 유사하지만 의미론적으로는 다름

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

In [ ]:
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.to('cuda:0')

* [CLS] token을 가져오고 [CLS] token의 embedding결과를 반환하는 함수

In [ ]:
def get_cls_token(sent_A):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=32
    ).to('cuda:0')
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

In [ ]:
print(get_cls_token("이순신은 조선 중기의 무신이다."))

* total_sent 문장 전부 embedding함
  * 전체 문장에 대해 각각의 vector 정보가 dict 형태로 저장

In [ ]:
total_sent_vector = {}
for i, sent in enumerate(total_sent):   # 전체 문장 pool을 전부 embedding!
    total_sent_vector[sent] = get_cls_token(sent)   # {key, value} = {문장, vector}
    if i % 500==0:
        print(i)

* `similar_sents` dictionary의 key값과 total_sent를 전부 비교하여 가장 유사도가 높은 문장을 가져옴

In [ ]:
import numpy as np

In [ ]:
def custom_cosine_similarity(a,b):
    numerator = np.dot(a,b.T)
    a_norm = np.sqrt(np.sum(a * a))
    b_norm = np.sqrt(np.sum(b * b, axis=-1))

    denominator = a_norm * b_norm
    return numerator/denominator

In [ ]:
non_similar_sents = {}

In [ ]:
for key in similar_sents.keys():    # similar_sents dict의 sentence를 가져옵니다.
    key_sent_vector = total_sent_vector[key]    # 전체 문장 pool에서 해당 sent의 vector을 가져옵니다.
    sentence_similarity = {}                    # 다음으로는 전체 문장 pool의 모든 vector와 비교하며
    for sent in total_sent:                     # 가장 유사한 문장을 가져옵니다.
        if sent not in similar_sents[key] and sent != key: # 9개의 문장에 해당하지 않는지 검사함
            sent_vector = total_sent_vector[sent]
            similarity = custom_cosine_similarity(key_sent_vector, sent_vector) # 9개의 문장에 해당하지 않으면 cosine similarity로 구함
            sentence_similarity[sent] = similarity
    sorted_sim = sorted(sentence_similarity.items(), key=lambda x: x[1], reverse=True)
    non_similar_sents[key] = sorted_sim[0:10]   # similar_sents dict의 문장과 가장 유사한 10개의 문장을 반환합니다. # TOP-N개 return

* `non_similar_sents` : 유사하지 않은 set

* 각 key에 대해 similarity가 가장 높은 10개 문장 확인
  * keyword가 유사하지만 의미론적으로 다름

In [ ]:
for i, key in enumerate(non_similar_sents.keys()):
    print('\n', key)
    for sent in non_similar_sents[key]:
        print("-", sent)
    if i > 3:
        break

'''
메일을 다 비울까 아니면 안읽은 것만 지울까?
- ('안방 말고 지금 거실 온도 좀 볼 수 있을까?', array([[0.98267]], dtype=float32))
- ('안 읽은 메일함이랑 스팸 메일함이랑 비교했을 때 어디가 더 차있지?', array([[0.97837853]], dtype=float32))
- ('가습기가 필요한게 아니고 제습기 하나 사야될 것 같지 않아?', array([[0.97624516]], dtype=float32))
- ('일월이 바쁘신가요, 아니면 이월이 더 바쁘신가요?', array([[0.97588336]], dtype=float32))
- ('안방하고 거실 중에 너가 로봇청소기를 틀고 싶은 곳은 어딜까?', array([[0.97562]], dtype=float32))
- ('안방 말고 거실 온도 보려면 어떻게 말해야하나?', array([[0.97547626]], dtype=float32))
- ('지금 네가 하고 싶은게 외출모드일까 아님 방범모드일까?', array([[0.9754139]], dtype=float32))
- ('메일을 상사에게 어떻게 보내야해?', array([[0.9753622]], dtype=float32))
- ('안방 말고 거실 지금 온도 보려면 뭐라고 해야해?', array([[0.9751789]], dtype=float32))
- ('목욕물을 개인별로 세팅하고 싶은데요 어떻게 하면 좋을까요?', array([[0.97513217]], dtype=float32))
'''

* 데이터 저장

In [ ]:
output = open('para_kqc_sim_data.txt', 'w', encoding='utf-8')   # 이걸 데이터로 만들어줍니다 :-)

In [ ]:
for i, key in enumerate(similar_sents.keys()):
    for sent in similar_sents[key]:
        output.write(key + '\t' + sent + '\t1\n')

In [ ]:
for i, key in enumerate(non_similar_sents.keys()):
    for sent in non_similar_sents[key]:
        output.write(key + '\t' + sent[0] + '\t0\n')

In [ ]:
output.close()